In [1]:
!pip install -q "git+https://github.com/Keyza-asyadda/DERMIS-MachineLearning.git#egg=dermis_utils&subdirectory=utils" pydot graphviz  

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import warnings
warnings.filterwarnings("ignore")

from tensorflow import keras
import tensorflow as tf

import matplotlib.pyplot as plt

import dermis_utils

import os
import gc

D0611 09:38:41.944201043      13 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0611 09:38:41.944229276      13 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0611 09:38:41.944232933      13 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0611 09:38:41.944235923      13 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0611 09:38:41.944238478      13 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0611 09:38:41.944247691      13 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0611 09:38:41.944250407      13 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0611 09:38:41.

In [3]:
DATA_DIR = '../input/isic-2019-capstone/ISIC_2019'
CACHE_DIR = '../tmp/'
SAVE_DIR = os.getcwd()
IMG_SIZE = 240
EPOCHS = 40
BATCH_SIZE = 128
BRIGHTNESS_LEVEL = .2
NUM_CLASSES = 8
LR = 0.00035

In [4]:
if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR)

In [5]:
try:
    tpu =tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    if len(tf.config.list_logical_devices("GPU")) > 1:
        strategy = tf.distribute.MirroredStrategy()
    else:
        strategy = tf.distribute.get_strategy()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU

## Import dataset

### Define augmentation function

In [6]:
if len(tf.config.list_logical_devices("TPU")) >0:
    augmentation_model = keras.Sequential([
        keras.layers.RandomBrightness(BRIGHTNESS_LEVEL),
        keras.layers.RandomFlip(),
        keras.layers.RandomRotation(.9),
    ])
else:
    with strategy.scope():
        augmentation_model = keras.Sequential([
            keras.layers.RandomBrightness(BRIGHTNESS_LEVEL),
            keras.layers.RandomFlip(),
            keras.layers.RandomRotation(.9),
        ])

@tf.function
def aug_image_label(images, labels):
    aug = augmentation_model(images, training = True)
    return aug, labels

In [7]:
train_ds = keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'train'),
    image_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = True,
    label_mode = "categorical"
)

Found 17731 files belonging to 8 classes.


In [8]:
train_non_aug = train_ds.prefetch(tf.data.AUTOTUNE)

In [9]:
class_weight = train_ds.class_names

In [10]:
# train_ds = train_ds.cache(os.path.join(CACHE_DIR, 'train'))
train_aug = train_ds.map(aug_image_label, num_parallel_calls = tf.data.AUTOTUNE, deterministic = False)
train_aug = train_aug.prefetch(tf.data.AUTOTUNE)

In [11]:
val_ds = keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'val'),
    image_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = False, # No need to shuffle validation set, and also to reduce computation
    label_mode = 'categorical'
)

Found 3800 files belonging to 8 classes.


In [12]:
val_ds = val_ds.cache(os.path.join(CACHE_DIR, 'val'))
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

## Calculate class weight for weighted cross entropy

In [13]:
class_weight = list(map(lambda x: len(os.listdir(os.path.join(DATA_DIR, 'train', x))), class_weight))
class_weight = tf.divide(tf.reduce_sum(class_weight), class_weight)
class_weight = tf.divide(class_weight * 10, tf.reduce_sum(class_weight))
class_weight = dict(enumerate(class_weight))

## Define model

In [14]:
def get_pretrained(
    trainable_blocks = [],
    train_all = False
):
    base_model = keras.applications.EfficientNetV2B1(
        include_top = False,
        input_shape = (IMG_SIZE, IMG_SIZE, 3),
        pooling = 'avg'
    )
    if not train_all:
        for layer in base_model.layers:
            layer.trainable = any(i in layer.name for i in trainable_blocks) and not isinstance(layer, keras.layers.BatchNormalization)
    else:
        for layer in base_model.layers:
            layer.trainable = not isinstance(layer, keras.layers.BatchNormalization)
    return base_model

In [15]:
def get_model(
    h_units = 64,
    trainable_blocks = ['top', 'block6i', 'block6h', 'block6g'],
    train_all_base_model = False
):
    base_model = get_pretrained(
        trainable_blocks = trainable_blocks,
        train_all = train_all_base_model
    )
    x = keras.layers.BatchNormalization()(base_model.output)
    x = keras.layers.Dense(h_units, activation = 'relu')(x)
    output = keras.layers.Dense(NUM_CLASSES)(x)
    return keras.Model(inputs = base_model.input, outputs = output)

## Train model

### Train model with Focal Loss

In [16]:
with strategy.scope():
    model = get_model()

28456008/28456008 [==============================] - 0s 0us/step


In [17]:
with strategy.scope():
    
    optimizer = keras.optimizers.Adam(
        LR
    )
    
    model.compile(
        optimizer = optimizer,
        loss = dermis_utils.losses.CategoricalFocalCrossentropy(gamma = 1., from_logits = True),
        metrics = [
            "accuracy",
            keras.metrics.AUC(
                multi_label = True,
                from_logits = True,
                num_labels = NUM_CLASSES
            ),
            keras.metrics.Recall(thresholds = 0),
            keras.metrics.Precision(thresholds = 0)
        ]
    )

In [18]:
with strategy.scope():
    
    csv_logger = keras.callbacks.CSVLogger(os.path.join(SAVE_DIR, 'aug_focal_loss_unfreezing.csv'))
    tensorboard = keras.callbacks.TensorBoard(os.path.join(SAVE_DIR, 'logs', 'aug_focal_loss_unfreezing/'))
    checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(SAVE_DIR, "aug-focal-loss-unfreezing-at-epoch-{epoch:02d}.hdf5"))
    reducelr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=.4, patience=4, min_lr=1e-7)

In [19]:
model.fit(
    train_aug,
    epochs = EPOCHS,
    callbacks = [csv_logger, tensorboard, checkpoint, reducelr],
    validation_data = val_ds
)

Epoch 1/40


2023-06-11 09:40:11.532486: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_123/ReadVariableOp.
2023-06-11 09:40:12.103656: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_123/ReadVariableOp.


139/139 [==============================] - ETA: 0s - loss: 0.9611 - accuracy: 0.5877 - auc: 0.7232 - recall: 0.8448 - precision: 0.2918

2023-06-11 09:41:06.290000: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-06-11 09:41:06.614001: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


139/139 [==============================] - 98s 383ms/step - loss: 0.9611 - accuracy: 0.5877 - auc: 0.7232 - recall: 0.8448 - precision: 0.2918 - val_loss: 0.8753 - val_accuracy: 0.6400 - val_auc: 0.8010 - val_recall: 0.9087 - val_precision: 0.3053 - lr: 3.5000e-04
Epoch 2/40
139/139 [==============================] - 21s 136ms/step - loss: 0.7349 - accuracy: 0.6453 - auc: 0.8137 - recall: 0.9110 - precision: 0.3119 - val_loss: 0.6511 - val_accuracy: 0.6905 - val_auc: 0.8381 - val_recall: 0.9324 - val_precision: 0.3050 - lr: 3.5000e-04
Epoch 3/40
139/139 [==============================] - 21s 134ms/step - loss: 0.6588 - accuracy: 0.6759 - auc: 0.8428 - recall: 0.9303 - precision: 0.3136 - val_loss: 0.6326 - val_accuracy: 0.6850 - val_auc: 0.8447 - val_recall: 0.9303 - val_precision: 0.3066 - lr: 3.5000e-04
Epoch 4/40
139/139 [==============================] - 21s 133ms/step - loss: 0.6018 - accuracy: 0.6982 - auc: 0.8563 - recall: 0.9377 - precision: 0.3174 - val_loss: 0.5902 - val_accu

In [20]:
model.save(os.path.join(SAVE_DIR,"aug_focal_loss_unfreezing"))

INFO:tensorflow:Assets written to: /kaggle/working/aug_focal_loss_unfreezing/assets


INFO:tensorflow:Assets written to: /kaggle/working/aug_focal_loss_unfreezing/assets


In [21]:
keras.utils.plot_model(
    model,
    to_file = os.path.join(SAVE_DIR, 'model.png')
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


### Train model with weighted cross entropy

In [22]:
keras.backend.clear_session()
gc.collect()

379563

In [23]:
with strategy.scope():
    model = get_model()

In [24]:
with strategy.scope():
    optimizer = keras.optimizers.Adam(
        LR
    )
    
    model.compile(
        optimizer = optimizer,
        loss = keras.losses.CategoricalCrossentropy(from_logits = True),
        metrics = [
            "accuracy",
            keras.metrics.AUC(
                multi_label = True,
                from_logits = True,
                num_labels = NUM_CLASSES
            ),
            keras.metrics.Recall(thresholds = 0),
            keras.metrics.Precision(thresholds = 0)
        ]
    )

In [25]:
with strategy.scope():
    csv_logger = keras.callbacks.CSVLogger(os.path.join(SAVE_DIR, 'aug_weighted_crossentropy_loss_unfreezing.csv'))
    tensorboard = keras.callbacks.TensorBoard(os.path.join(SAVE_DIR, 'logs', 'aug_weighted_crossentropy_loss_unfreezing/'))
    checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(SAVE_DIR, "aug-weighted-crossentropy-loss-unfreezing-at-epoch-{epoch:02d}.hdf5"))
    reducelr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=.4, patience=4, min_lr=1e-7)

In [26]:
model.fit(
    train_aug,
    epochs = EPOCHS,
    callbacks = [csv_logger, tensorboard, checkpoint, reducelr],
    class_weight = class_weight,
    validation_data = val_ds
)

Epoch 1/40


2023-06-11 09:57:31.519501: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_123/ReadVariableOp.
2023-06-11 09:57:32.203979: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_123/ReadVariableOp.


139/139 [==============================] - ETA: 0s - loss: 0.4807 - accuracy: 0.3333 - auc: 0.7274 - recall: 0.7944 - precision: 0.1944

2023-06-11 09:58:27.912136: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-06-11 09:58:28.305900: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


139/139 [==============================] - 100s 368ms/step - loss: 0.4807 - accuracy: 0.3333 - auc: 0.7274 - recall: 0.7944 - precision: 0.1944 - val_loss: 1.5091 - val_accuracy: 0.4355 - val_auc: 0.7994 - val_recall: 0.9047 - val_precision: 0.2092 - lr: 3.5000e-04
Epoch 2/40
139/139 [==============================] - 21s 139ms/step - loss: 0.3850 - accuracy: 0.4519 - auc: 0.8031 - recall: 0.8780 - precision: 0.2137 - val_loss: 1.2747 - val_accuracy: 0.5350 - val_auc: 0.8375 - val_recall: 0.9208 - val_precision: 0.2264 - lr: 3.5000e-04
Epoch 3/40
139/139 [==============================] - 21s 134ms/step - loss: 0.3385 - accuracy: 0.5127 - auc: 0.8320 - recall: 0.9095 - precision: 0.2198 - val_loss: 1.1682 - val_accuracy: 0.5708 - val_auc: 0.8606 - val_recall: 0.9503 - val_precision: 0.2239 - lr: 3.5000e-04
Epoch 4/40
139/139 [==============================] - 21s 135ms/step - loss: 0.3111 - accuracy: 0.5296 - auc: 0.8568 - recall: 0.9314 - precision: 0.2228 - val_loss: 1.2155 - val_acc

In [27]:
model.save(os.path.join(SAVE_DIR,"aug_weighted_crossentropy_loss_unfreezing"))

INFO:tensorflow:Assets written to: /kaggle/working/aug_weighted_crossentropy_loss_unfreezing/assets


INFO:tensorflow:Assets written to: /kaggle/working/aug_weighted_crossentropy_loss_unfreezing/assets


### Train focal loss model without augmentation

In [28]:
keras.backend.clear_session()
gc.collect()

378905

In [29]:
with strategy.scope():
    model = get_model()

In [30]:
with strategy.scope():
    optimizer = keras.optimizers.Adam(
        LR
    )
    
    model.compile(
        optimizer = optimizer,
        loss = dermis_utils.losses.CategoricalFocalCrossentropy(gamma = 1., from_logits = True),
        metrics = [
            "accuracy",
            keras.metrics.AUC(
                multi_label = True,
                from_logits = True,
                num_labels = NUM_CLASSES
            ),
            keras.metrics.Recall(thresholds = 0),
            keras.metrics.Precision(thresholds = 0)
        ]
    )

In [31]:
with strategy.scope():
    
    csv_logger = keras.callbacks.CSVLogger(os.path.join(SAVE_DIR, 'non_aug_focal_loss_unfreezing.csv'))
    tensorboard = keras.callbacks.TensorBoard(os.path.join(SAVE_DIR, 'logs', 'non_aug_focal_loss_unfreezing/'))
    checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(SAVE_DIR, "non-aug-focal-loss-unfreezing-at-epoch-{epoch:02d}.hdf5"))
    reducelr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=.4, patience=4, min_lr=1e-7)

In [32]:
model.fit(
    train_non_aug,
    epochs = EPOCHS,
    callbacks = [csv_logger, tensorboard, checkpoint, reducelr],
    validation_data = val_ds
)

Epoch 1/40


2023-06-11 10:14:57.570010: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_123/ReadVariableOp.
2023-06-11 10:14:58.209565: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_123/ReadVariableOp.


139/139 [==============================] - ETA: 0s - loss: 0.9413 - accuracy: 0.5913 - auc: 0.7489 - recall: 0.8764 - precision: 0.2972

2023-06-11 10:15:43.728118: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-06-11 10:15:44.072093: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


139/139 [==============================] - 89s 384ms/step - loss: 0.9413 - accuracy: 0.5913 - auc: 0.7489 - recall: 0.8764 - precision: 0.2972 - val_loss: 0.8164 - val_accuracy: 0.6666 - val_auc: 0.8341 - val_recall: 0.9216 - val_precision: 0.3309 - lr: 3.5000e-04
Epoch 2/40
139/139 [==============================] - 15s 104ms/step - loss: 0.6725 - accuracy: 0.6755 - auc: 0.8451 - recall: 0.9385 - precision: 0.3152 - val_loss: 0.6510 - val_accuracy: 0.6961 - val_auc: 0.8672 - val_recall: 0.9363 - val_precision: 0.3331 - lr: 3.5000e-04
Epoch 3/40
139/139 [==============================] - 15s 103ms/step - loss: 0.5789 - accuracy: 0.7077 - auc: 0.8780 - recall: 0.9553 - precision: 0.3204 - val_loss: 0.5786 - val_accuracy: 0.7118 - val_auc: 0.8871 - val_recall: 0.9571 - val_precision: 0.3183 - lr: 3.5000e-04
Epoch 4/40
139/139 [==============================] - 15s 102ms/step - loss: 0.5114 - accuracy: 0.7362 - auc: 0.9003 - recall: 0.9663 - precision: 0.3267 - val_loss: 0.5395 - val_accu

In [33]:
model.save(os.path.join(SAVE_DIR, 'non_aug_focal_loss_unfreezing'))

INFO:tensorflow:Assets written to: /kaggle/working/non_aug_focal_loss_unfreezing/assets


INFO:tensorflow:Assets written to: /kaggle/working/non_aug_focal_loss_unfreezing/assets


### Train weighted cross entropy model without augmentation

In [34]:
keras.backend.clear_session()
gc.collect()

378905

In [35]:
with strategy.scope():
    model = get_model()

In [36]:
with strategy.scope():
    optimizer = keras.optimizers.Adam(
        LR
    )
    
    model.compile(
        optimizer = optimizer,
        loss = keras.losses.CategoricalCrossentropy(from_logits = True),
        metrics = [
            "accuracy",
            keras.metrics.AUC(
                multi_label = True,
                from_logits = True,
                num_labels = NUM_CLASSES
            ),
            keras.metrics.Recall(thresholds = 0),
            keras.metrics.Precision(thresholds = 0)
        ]
    )

In [37]:
with strategy.scope():
    
    csv_logger = keras.callbacks.CSVLogger(os.path.join(SAVE_DIR, 'non_aug_weighted_crossentropy_loss_unfreezing.csv'))
    tensorboard = keras.callbacks.TensorBoard(os.path.join(SAVE_DIR, 'logs', 'non_aug_weighted_crossentropy_loss_unfreezing/'))
    checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(SAVE_DIR, "non-aug-weighted-crossentropy-loss-unfreezing-at-epoch-{epoch:02d}.hdf5"))
    reducelr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=.4, patience=4, min_lr=1e-7)

In [38]:
model.fit(
    train_non_aug,
    epochs = EPOCHS,
    class_weight = class_weight,
    callbacks = [csv_logger, tensorboard, checkpoint, reducelr],
    validation_data = val_ds
)

Epoch 1/40


2023-06-11 10:28:06.830095: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_123/ReadVariableOp.
2023-06-11 10:28:07.463228: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_123/ReadVariableOp.


139/139 [==============================] - ETA: 0s - loss: 0.4732 - accuracy: 0.3567 - auc: 0.7420 - recall: 0.8103 - precision: 0.1905

2023-06-11 10:28:52.570697: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-06-11 10:28:52.966957: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


139/139 [==============================] - 89s 372ms/step - loss: 0.4732 - accuracy: 0.3567 - auc: 0.7420 - recall: 0.8103 - precision: 0.1905 - val_loss: 1.5786 - val_accuracy: 0.4505 - val_auc: 0.8228 - val_recall: 0.8784 - val_precision: 0.2017 - lr: 3.5000e-04
Epoch 2/40
139/139 [==============================] - 14s 99ms/step - loss: 0.3427 - accuracy: 0.5027 - auc: 0.8438 - recall: 0.9075 - precision: 0.2158 - val_loss: 1.2267 - val_accuracy: 0.5463 - val_auc: 0.8621 - val_recall: 0.9400 - val_precision: 0.2233 - lr: 3.5000e-04
Epoch 3/40
139/139 [==============================] - 14s 99ms/step - loss: 0.2874 - accuracy: 0.5627 - auc: 0.8765 - recall: 0.9381 - precision: 0.2276 - val_loss: 1.0589 - val_accuracy: 0.6092 - val_auc: 0.8855 - val_recall: 0.9597 - val_precision: 0.2321 - lr: 3.5000e-04
Epoch 4/40
139/139 [==============================] - 16s 111ms/step - loss: 0.2545 - accuracy: 0.5989 - auc: 0.8908 - recall: 0.9536 - precision: 0.2319 - val_loss: 1.0284 - val_accura

In [39]:
model.save(os.path.join(SAVE_DIR, 'non_aug_weighted_crossentropy_loss_unfreezing'))

INFO:tensorflow:Assets written to: /kaggle/working/non_aug_weighted_crossentropy_loss_unfreezing/assets


INFO:tensorflow:Assets written to: /kaggle/working/non_aug_weighted_crossentropy_loss_unfreezing/assets
